<a href="https://colab.research.google.com/github/Fabchirajoul/Transformer_MLM_AND_BINARY_CLASSIFICATION/blob/main/DistilRoberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install accelerate -U

In [2]:
# pip install transformers

In [3]:
# pip install datasets

In [4]:
import random

# For our testing dataset

def random_sample_testing(input_file, output_file, sample_percentage=0.2):
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Extract header and content separately
    header = lines[0]
    content = lines[1:]

    num_lines = len(content)
    sample_size = int(num_lines * sample_percentage)

    sampled_lines = random.sample(content, sample_size)

    # Add the "TEXT" header back at the beginning
    sampled_lines = [header] + sampled_lines

    with open(output_file, 'w', encoding='utf-8') as file:
        file.writelines(sampled_lines)

# Example usage
input_document = "/content/classification_dataset.csv"
output_sample = "/content/testing_dataset_classification.csv"
random_sample_testing(input_document, output_sample, sample_percentage=0.2)



# For our training dataset

def random_sample_testing(input_file, output_file, sample_percentage=0.7):
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Extract header and content separately
    header = lines[0]
    content = lines[1:]

    num_lines = len(content)
    sample_size = int(num_lines * sample_percentage)

    sampled_lines = random.sample(content, sample_size)

    # Add the "TEXT" header back at the beginning
    sampled_lines = [header] + sampled_lines

    with open(output_file, 'w', encoding='utf-8') as file:
        file.writelines(sampled_lines)

# Example usage
input_document = "/content/classification_dataset.csv"
output_sample = "/content/training_dataset_classification.csv"
random_sample_testing(input_document, output_sample, sample_percentage=0.7)


# For our validation dataset


def random_sample_testing(input_file, output_file, sample_percentage=0.1):
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Extract header and content separately
    header = lines[0]
    content = lines[1:]

    num_lines = len(content)
    sample_size = int(num_lines * sample_percentage)

    sampled_lines = random.sample(content, sample_size)

    # Add the "TEXT" header back at the beginning
    sampled_lines = [header] + sampled_lines

    with open(output_file, 'w', encoding='utf-8') as file:
        file.writelines(sampled_lines)

# Example usage
input_document = "/content/classification_dataset.csv"
output_sample = "/content/validation_dataset_classification.csv"
random_sample_testing(input_document, output_sample, sample_percentage=0.1)

# LOADING THE DATASET

In [5]:
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import  pandas as pd
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

import random
import torch
import math



data_files = {"train": "/content/training_dataset_classification.csv",
              "validation": "/content/validation_dataset_classification.csv",
              "test": "/content/testing_dataset_classification.csv"
              }
df_DistilRoberta = load_dataset("csv", data_files=data_files)
df_DistilRoberta

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 277
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 39
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 79
    })
})

#  REQUIRED MODELS TO BE BUILT WHICH WILL BE USED FOR COMPARISON WITH OUR BASE MODEL

# 1.Climate Bert D+s
# 2.Climate Bert D
# 3.Climate Bert S
# 4.Climate Bert F
# 5.DistilROBERTa

In [6]:
labels_train = df_DistilRoberta['train']
labels_test = df_DistilRoberta['test']

print(labels_train)
print(labels_test)

Dataset({
    features: ['text', 'label'],
    num_rows: 277
})
Dataset({
    features: ['text', 'label'],
    num_rows: 79
})


# Tokenization for the different Models Required for Comparison

In [7]:
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          Trainer,
                          TrainingArguments)

tokenizer_DistilROBERTa = AutoTokenizer.from_pretrained("distilroberta-base")
tokenizer_climteBertd_s = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-d-s")
tokenizer_climteBert_s = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-s")
tokenizer_climteBert_d = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-d")
tokenizer_climteBert_f = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")





# Defining a generic tokenization function to be used in tokenizing the dataset based on the different pretrained model
tokenizer_DistilROBERTa_func = lambda sentences: tokenizer_DistilROBERTa(sentences['text'], \
                                            padding="max_length", \
                                            truncation=True)

# Defining a generic tokenization function to be used in tokenizing the dataset based on the different pretrained model
tokenizer_climteBertd_s_func = lambda sentences: tokenizer_climteBertd_s(sentences['text'], \
                                            padding=True, \
                                            truncation=True, \
                                            return_tensors='pt')

# Defining a generic tokenization function to be used in tokenizing the dataset based on the different pretrained model
tokenizer_climteBert_s_func = lambda sentences: tokenizer_climteBert_s(sentences['text'], \
                                            padding="max_length", \
                                            truncation=True, \
                                            return_tensors='pt')

# Defining a generic tokenization function to be used in tokenizing the dataset based on the different pretrained model
tokenizer_climteBert_d_func = lambda sentences: tokenizer_climteBert_d(sentences['text'], \
                                           padding="max_length", \
                                            truncation=True, \
                                            return_tensors='pt')

# Defining a generic tokenization function to be used in tokenizing the dataset based on the different pretrained model
tokenizer_climteBert_f_func = lambda sentences: tokenizer_climteBert_f(sentences['text'], \
                                           padding="max_length", \
                                            truncation=True, \
                                            return_tensors='pt')





# Tokenizing the dataset dictionary using the tokenizer for the different pretained model

# 1. DistilROBERTa Tokenizer
tokenized_distilRobert = df_DistilRoberta.map(tokenizer_DistilROBERTa_func, batched=True)


# 2. ClimateBert D-S Tokenizer
tokenizer_climteBertd_s = df_DistilRoberta.map(tokenizer_climteBertd_s_func, batched=True)


# 3. ClimateBert S Tokenizer
tokenizer_climteBert_s = df_DistilRoberta.map(tokenizer_climteBert_s_func, batched=True)


# 4. ClimateBert D Tokenizer
tokenizer_climteBert_d = df_DistilRoberta.map(tokenizer_climteBert_d_func, batched=True)




# 5. ClimateBert F Tokenizer
tokenizer_climteBert_f = df_DistilRoberta.map(tokenizer_climteBert_f_func, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

# Viewing the tokenized data

In [8]:
print("======================================================================================================")
print("Tokenized DistilROBERTa:  \n\n\n", tokenized_distilRobert)
print("======================================================================================================")
print("Tokenized ClimateBertd+s:  \n\n\n", tokenizer_climteBertd_s)
print("======================================================================================================")
print("Tokenized ClimateBerts:  \n\n\n", tokenizer_climteBert_s)
print("======================================================================================================")
print("Tokenized ClimateBertd:  \n\n\n", tokenizer_climteBert_d)
print("======================================================================================================")
print("Tokenized ClimateBertf:  \n\n\n", tokenizer_climteBert_f)
print("======================================================================================================")

Tokenized DistilROBERTa:  


 DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 277
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 39
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 79
    })
})
Tokenized ClimateBertd+s:  


 DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 277
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 39
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 79
    })
})
Tokenized ClimateBerts:  


 DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 277
    })
    validation: Dataset({
      

# Removing columns which the machine will not understand

In [9]:
# Machine does not understand categorical data i.e. text so we need to remove it.
columns_to_remove_classification = ['text']


# Remove the specified columns from each split for DistilROBERTa
for split in tokenized_distilRobert.keys():
    tokenized_distilRobert[split] = tokenized_distilRobert[split].remove_columns(columns_to_remove_classification)


# Remove the specified columns from each split for ClimateBert d+s
for split in tokenizer_climteBertd_s.keys():
    tokenizer_climteBertd_s[split] = tokenizer_climteBertd_s[split].remove_columns(columns_to_remove_classification)

# Remove the specified columns from each split for ClimateBert s
for split in tokenizer_climteBert_s.keys():
    tokenizer_climteBert_s[split] = tokenizer_climteBert_s[split].remove_columns(columns_to_remove_classification)


# Remove the specified columns from each split for ClimateBert d
for split in tokenizer_climteBert_d.keys():
    tokenizer_climteBert_d[split] = tokenizer_climteBert_d[split].remove_columns(columns_to_remove_classification)

# Remove the specified columns from each split for ClimateBert f
for split in tokenizer_climteBert_f.keys():
    tokenizer_climteBert_f[split] = tokenizer_climteBert_f[split].remove_columns(columns_to_remove_classification)

# CONVERTING LABEL TO LABELS

In [10]:
# Pretrained model does not understand the column name label so we ought to convert it to labels so it can understand

# converting the column name label to labels since that is what the machine understands for a transformer model
tokenized_distilRobert = tokenized_distilRobert.rename_column("label", "labels")
tokenizer_climteBertd_s = tokenizer_climteBertd_s.rename_column("label", "labels")
tokenizer_climteBert_s = tokenizer_climteBert_s.rename_column("label", "labels")
tokenizer_climteBert_d = tokenizer_climteBert_d.rename_column("label", "labels")
tokenizer_climteBert_f = tokenizer_climteBert_f.rename_column("label", "labels")


# VIEWING WHAT WE DROPPED

In [11]:
print("======================================================================================================")
print("Tokenized DistilROBERTa:  \n\n\n", tokenized_distilRobert)
print("======================================================================================================")
print("Tokenized ClimateBertd+s:  \n\n\n", tokenizer_climteBertd_s)
print("======================================================================================================")
print("Tokenized ClimateBerts:  \n\n\n", tokenizer_climteBert_s)
print("======================================================================================================")
print("Tokenized ClimateBertd:  \n\n\n", tokenizer_climteBert_d)
print("======================================================================================================")
print("Tokenized ClimateBertf:  \n\n\n", tokenizer_climteBert_f)
print("======================================================================================================")

Tokenized DistilROBERTa:  


 DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 277
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 39
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 79
    })
})
Tokenized ClimateBertd+s:  


 DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 277
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 39
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 79
    })
})
Tokenized ClimateBerts:  


 DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 277
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_ma

# SETTING A PYTORCH DATA FOR OUR DATA LOADER

In [12]:
tokenized_distilRobert.set_format("torch")
tokenizer_climteBertd_s.set_format("torch")
tokenizer_climteBert_s.set_format("torch")
tokenizer_climteBert_d.set_format("torch")
tokenizer_climteBert_f.set_format("torch")


print(tokenized_distilRobert.column_names)
print(tokenized_distilRobert['train'][0])

{'train': ['labels', 'input_ids', 'attention_mask'], 'validation': ['labels', 'input_ids', 'attention_mask'], 'test': ['labels', 'input_ids', 'attention_mask']}
{'labels': tensor(1), 'input_ids': tensor([    0,   170,    33,   341,     5,   208,  3134, 25251,   468,   398,
            4,   288,   577,    15,  4662, 29523,  2580,   998,     7,  2179,
           84,  4761,  1385,    36,   250,  3297,   134,    43,     8,   251,
         1385,    36,   250,  3297,   176,    43, 13944,   565,     4,  1437,
          166,    32,  2021,     7,  9499,  4363, 18755,   420,    84,  1414,
           30, 12060,   634,    10,  1161,  4276,  1548,     6,     8,     7,
        12432,   154,    84,  2111,  6946,  2819,     7,  1161,  4276,  4363,
         5035,    30,   291,  1749,    36,  5632,     5,  8219,     9,  6300,
            6,    61,    52,    32, 12432,   154,     7,     5,  3445,   168,
           18, 11001,  5828,  5427, 37500,    31,    84,  5008, 15472,  2720,
            8, 12831,   

# DATALOADER

In [13]:
# Check the expected input size from the model's configuration
expected_batch_size = 8


# Data Collator
data_collator_distilRobert = DataCollatorWithPadding(tokenizer=tokenizer_DistilROBERTa)

# 1. Adjust the DataLoader batch size to match the expected size for DistilROBERTa
train_dataloader_distilRobert = DataLoader(
    tokenized_distilRobert["train"],
    shuffle=True,
    batch_size=expected_batch_size,
    collate_fn=data_collator_distilRobert
)
eval_dataloader_distilRobert = DataLoader(
    tokenized_distilRobert["validation"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_distilRobert
)

test_dataloader_distilRobert = DataLoader(
    tokenized_distilRobert["test"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_distilRobert
)

# Data Collator
data_collator_climteBertd_s = DataCollatorWithPadding(tokenizer=tokenizer_climteBertd_s)

# 2. Adjust the DataLoader batch size to match the expected size for climteBertd_s
train_dataloader_climteBertd_s = DataLoader(
    tokenizer_climteBertd_s["train"],
    shuffle=True,
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBertd_s
)
eval_dataloader_climteBertd_s = DataLoader(
    tokenizer_climteBertd_s["validation"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBertd_s
)

test_dataloader_climteBertd_s = DataLoader(
    tokenizer_climteBertd_s["test"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBertd_s
)


# Data Collator
data_collator_climteBert_s = DataCollatorWithPadding(tokenizer=tokenizer_climteBert_s)


# 3. Adjust the DataLoader batch size to match the expected size for climteBert_s
train_dataloader_climteBert_s = DataLoader(
    tokenizer_climteBert_s["train"],
    shuffle=True,
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBert_s
)
eval_dataloader_climteBert_s = DataLoader(
    tokenizer_climteBert_s["validation"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBert_s
)

test_dataloader_climteBertd_s = DataLoader(
    tokenizer_climteBert_s["test"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBert_s
)


# Data Collator
data_collator_climteBert_d = DataCollatorWithPadding(tokenizer=tokenizer_climteBert_d)


# 4. Adjust the DataLoader batch size to match the expected size for _climteBert_d
train_dataloader_climteBert_d = DataLoader(
    tokenizer_climteBert_d["train"],
    shuffle=True,
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBert_d
)
eval_dataloader_climteBert_d = DataLoader(
    tokenizer_climteBert_d["validation"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBert_d
)

test_dataloader_climteBert_d = DataLoader(
    tokenizer_climteBert_d["test"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBert_d
)


# Data Collator
data_collator_climteBert_f = DataCollatorWithPadding(tokenizer=tokenizer_climteBert_f)


# 5. Adjust the DataLoader batch size to match the expected size for _climteBert_d
train_dataloader_climteBert_f = DataLoader(
    tokenizer_climteBert_f["train"],
    shuffle=True,
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBert_f
)
eval_dataloader_climteBert_f = DataLoader(
    tokenizer_climteBert_f["validation"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBert_f
)

test_dataloader_climteBert_f = DataLoader(
    tokenizer_climteBert_f["test"],
    batch_size=expected_batch_size,
    collate_fn=data_collator_climteBert_f
)

# Defining the models

In [14]:
# Invoking the different models from the pretrained directory in hugging face

model_DistilRoberta = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=2)
model_ClimateBert_D_S = AutoModelForMaskedLM.from_pretrained("climatebert/distilroberta-base-climate-d-s", num_labels=2)
model_ClimateBertS = AutoModelForMaskedLM.from_pretrained("climatebert/distilroberta-base-climate-s", num_labels=2)
model_ClimateBert_D = AutoModelForMaskedLM.from_pretrained("climatebert/distilroberta-base-climate-d", num_labels=2)
model_ClimateBert_F = AutoModelForMaskedLM.from_pretrained("climatebert/distilroberta-base-climate-f", num_labels=2)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

# Passing trained batch to our model

In [15]:
from transformers import default_data_collator


# 1. Checking to make sure the batch fits to our model

for batch in train_dataloader_distilRobert:
    break
{k: v.shape for k, v in batch.items()}




You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512])}